In [1]:
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import plotly.offline as py


In [3]:
spark = SparkSession.builder.appName("flights").getOrCreate()
import json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType
with open("../../util/schema.json","r") as f:
    schema = StructType.fromJson(json.load(f))

In [4]:
df = spark.read.csv("../../data.nosync/cleaned/cleaned_flights.csv",schema=schema, header=True)

In [5]:
# create a new column 'label' that is 1 if the flight is delayed and 0 if it is not
df = df.withColumn("label", when(df["ArrDelay"] > 0, 1).otherwise(0))

In [6]:
features = [
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'Reporting_Airline',
 'Origin',
 'Dest',
 #'DepDelay',
 'CRSDepTime',
 'CRSArrTime',
 'CRSElapsedTime',
 'AirTime',
 'Distance',
 'ORIGIN_STATE',
 'DEST_STATE',
 'label'
 ]


# mantain only the features in features list
df = df.select(features)

In [7]:
df.show(10)

+-------+-----+----------+---------+-----------------+------+----+----------+----------+--------------+-------+--------+------------+----------+-----+
|Quarter|Month|DayofMonth|DayOfWeek|Reporting_Airline|Origin|Dest|CRSDepTime|CRSArrTime|CRSElapsedTime|AirTime|Distance|ORIGIN_STATE|DEST_STATE|label|
+-------+-----+----------+---------+-----------------+------+----+----------+----------+--------------+-------+--------+------------+----------+-----+
|      4|   10|         4|        5|               9E|   ATL| OMA|       855|      1025|         150.0|  124.0|   821.0|          GA|        NE|    0|
|      4|   10|        26|        6|               9E|   IAH| MSP|       730|      1021|         171.0|  142.0|  1034.0|          TX|        MN|    0|
|      4|   10|        12|        6|               9E|   LGA| CLT|      1625|      1839|         134.0|   81.0|   544.0|          NY|        NC|    1|
|      4|   10|         4|        5|               9E|   OMA| ATL|      1100|      1409|      

In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.feature import OneHotEncoder

In [9]:
df.dtypes

[('Quarter', 'int'),
 ('Month', 'int'),
 ('DayofMonth', 'int'),
 ('DayOfWeek', 'int'),
 ('Reporting_Airline', 'string'),
 ('Origin', 'string'),
 ('Dest', 'string'),
 ('CRSDepTime', 'int'),
 ('CRSArrTime', 'int'),
 ('CRSElapsedTime', 'double'),
 ('AirTime', 'double'),
 ('Distance', 'double'),
 ('ORIGIN_STATE', 'string'),
 ('DEST_STATE', 'string'),
 ('label', 'int')]

In [10]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

categoricalCols = [field for (field, dataType) in df.dtypes
if dataType == "string"]

indexOutputCols = [x + "Index" for x in categoricalCols]
oheOutputCols = [x + "OHE" for x in categoricalCols]

stringIndexer = StringIndexer(inputCols=categoricalCols,
                                outputCols=indexOutputCols,
                                handleInvalid="skip")
oheEncoder = OneHotEncoder(inputCols=indexOutputCols,
                            outputCols=oheOutputCols)

numericCols = [field for (field, dataType) in df.dtypes
                    if ((dataType == "double" or dataType == "int" ) & (field != "label"))]

assemblerInputs = oheOutputCols + numericCols

vecAssembler = VectorAssembler(inputCols=assemblerInputs,
                        outputCol="features")


In [11]:
# create the pipeline
pipeline = Pipeline(stages=[stringIndexer, oheEncoder, vecAssembler])

# fit the pipeline to the data
pipelineModel = pipeline.fit(df)

# transform the data
df_proc = pipelineModel.transform(df)

In [12]:
# select the features and label columns
df_proc = df_proc.select("features","label")


In [13]:
# show the first 5 rows
df_proc.show(5)


22/12/27 18:10:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+--------------------+-----+
|            features|label|
+--------------------+-----+
|(761,[8,15,389,64...|    0|
|(761,[8,20,340,64...|    0|
|(761,[8,33,339,64...|    1|
|(761,[8,75,330,67...|    1|
|(761,[8,94,330,66...|    0|
+--------------------+-----+
only showing top 5 rows



In [14]:
# count the number of rows with label 1
df_proc.filter(df_proc.label == 1).count()

2501251

In [15]:
# count the number of rows with label 0
df_proc.filter(df_proc.label == 0).count()

3745488

In [16]:
sample_rate = 2501251/3745488
sample_rate

0.6678037681605175

In [17]:
# split the data into train and test
train, test = df_proc.randomSplit([0.8, 0.2], seed=42)


In [18]:

train_1 = train.filter(train.label == 1)


train_0 = train.filter(train.label == 0).sample(False, sample_rate, seed=42)


# merge the two datasets
train = train_1.union(train_0)

In [19]:
# count the number of rows with label 1
train.filter(train.label == 1).count()

2001882

In [20]:
# count the number of rows with label 0
train.filter(train.label == 0).count()

2002297

In [21]:
# create the model
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=30,maxDepth=10)


In [22]:
# train the model
model = rf.fit(train)

22/12/27 18:15:06 WARN MemoryStore: Not enough space to cache rdd_95_6 in memory! (computed 13.9 MiB so far)
22/12/27 18:15:06 WARN MemoryStore: Not enough space to cache rdd_95_2 in memory! (computed 21.2 MiB so far)
22/12/27 18:15:06 WARN BlockManager: Persisting block rdd_95_6 to disk instead.
22/12/27 18:15:06 WARN BlockManager: Persisting block rdd_95_2 to disk instead.
22/12/27 18:15:06 WARN MemoryStore: Not enough space to cache rdd_95_5 in memory! (computed 21.2 MiB so far)
22/12/27 18:15:06 WARN BlockManager: Persisting block rdd_95_5 to disk instead.
22/12/27 18:15:06 WARN MemoryStore: Not enough space to cache rdd_95_3 in memory! (computed 13.9 MiB so far)
22/12/27 18:15:06 WARN BlockManager: Persisting block rdd_95_3 to disk instead.
22/12/27 18:15:06 WARN MemoryStore: Not enough space to cache rdd_95_4 in memory! (computed 21.2 MiB so far)
22/12/27 18:15:06 WARN BlockManager: Persisting block rdd_95_4 to disk instead.
22/12/27 18:15:06 WARN MemoryStore: Not enough space to

22/12/27 18:15:18 WARN MemoryStore: Not enough space to cache rdd_95_8 in memory! (computed 72.4 MiB so far)
22/12/27 18:15:18 WARN BlockManager: Persisting block rdd_95_8 to disk instead.
22/12/27 18:15:18 WARN MemoryStore: Not enough space to cache rdd_95_10 in memory! (computed 13.9 MiB so far)
22/12/27 18:15:18 WARN BlockManager: Persisting block rdd_95_10 to disk instead.
22/12/27 18:15:18 WARN MemoryStore: Not enough space to cache rdd_95_11 in memory! (computed 8.8 MiB so far)
22/12/27 18:15:18 WARN BlockManager: Persisting block rdd_95_11 to disk instead.
22/12/27 18:15:18 WARN MemoryStore: Not enough space to cache rdd_95_9 in memory! (computed 72.4 MiB so far)
22/12/27 18:15:18 WARN BlockManager: Persisting block rdd_95_9 to disk instead.
22/12/27 18:15:21 WARN MemoryStore: Not enough space to cache rdd_95_9 in memory! (computed 171.6 MiB so far)
22/12/27 18:15:21 WARN MemoryStore: Not enough space to cache rdd_95_8 in memory! (computed 31.9 MiB so far)
22/12/27 18:15:22 WARN

22/12/27 18:15:28 WARN MemoryStore: Not enough space to cache rdd_95_14 in memory! (computed 72.4 MiB so far)
22/12/27 18:15:28 WARN MemoryStore: Not enough space to cache rdd_95_15 in memory! (computed 171.6 MiB so far)
22/12/27 18:15:30 WARN MemoryStore: Not enough space to cache rdd_95_13 in memory! (computed 257.4 MiB so far)
22/12/27 18:15:31 WARN MemoryStore: Not enough space to cache rdd_95_12 in memory! (computed 47.9 MiB so far)
22/12/27 18:15:32 WARN MemoryStore: Not enough space to cache rdd_95_18 in memory! (computed 31.9 MiB so far)
22/12/27 18:15:32 WARN BlockManager: Persisting block rdd_95_18 to disk instead.


22/12/27 18:15:34 WARN MemoryStore: Not enough space to cache rdd_95_16 in memory! (computed 112.4 MiB so far)
22/12/27 18:15:34 WARN BlockManager: Persisting block rdd_95_16 to disk instead.
22/12/27 18:15:34 WARN MemoryStore: Not enough space to cache rdd_95_17 in memory! (computed 112.4 MiB so far)
22/12/27 18:15:34 WARN BlockManager: Persisting block rdd_95_17 to disk instead.
22/12/27 18:15:34 WARN MemoryStore: Not enough space to cache rdd_95_19 in memory! (computed 112.4 MiB so far)
22/12/27 18:15:34 WARN BlockManager: Persisting block rdd_95_19 to disk instead.


22/12/27 18:15:36 WARN MemoryStore: Not enough space to cache rdd_95_18 in memory! (computed 257.4 MiB so far)


...

22/12/27 18:19:54 WARN MemoryStore: Not enough space to cache rdd_95_28 in memory! (computed 72.4 MiB so far)
22/12/27 18:19:54 WARN MemoryStore: Not enough space to cache rdd_95_29 in memory! (computed 1067.6 KiB so far)


22/12/27 18:19:55 WARN MemoryStore: Not enough space to cache rdd_95_30 in memory! (computed 1067.6 KiB so far)
22/12/27 18:19:55 WARN MemoryStore: Not enough space to cache rdd_95_31 in memory! (computed 1067.6 KiB so far)


22/12/27 18:19:55 WARN MemoryStore: Not enough space to cache rdd_95_32 in memory! (computed 72.4 MiB so far)
22/12/27 18:19:56 WARN MemoryStore: Not enough space to cache rdd_95_33 in memory! (computed 1067.6 KiB so far)


22/12/27 18:19:57 WARN MemoryStore: Not enough space to cache rdd_95_34 in memory! (computed 1067.6 KiB so far)


22/12/27 18:19:58 WARN MemoryStore: Not enough space to cache rdd_95_35 in memory! (computed 72.4 MiB so far)


In [24]:
# evaluate the model
predictions = model.transform(train)

# evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

# print the accuracy
print("Accuracy = %g " % accuracy)



Test Error = 0.414965 
Accuracy = 0.585035 


Test Error = 0.419311 
Accuracy = 0.580689 